In [1]:
pip install selenium


You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.2.8/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install undetected-chromedriver

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.2.8/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bs4

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.2.8/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver-manager

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.2.8/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


#### Imports

In [1]:
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import pandas as pd
import re
import statistics

import undetected_chromedriver as uc



In [2]:
pd.set_option('display.max_columns', None)


#### Create driver and open webpage

In [3]:
driver = uc.Chrome()

driver.implicitly_wait(5)
driver.get('https://www.fotocasa.es/es/')
driver.maximize_window()


#### Accept cookies and find apartments by district

In [4]:
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar y cerrar')]"))).click()
time.sleep(1)
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomInput-input sui-AtomInput-input-size-m'.replace(' ', '.')))).click()
time.sleep(1)


barrios=["san esteban distrito leon "]

time.sleep(1)
buscar= driver.find_element(by=By.CLASS_NAME,value="sui-AtomInput-input sui-AtomInput-input-size-m".replace(' ', '.'))
buscar.send_keys(barrios[0])
time.sleep(3)

WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Buscar')]"))).click()



#### Scroll through the document (making sure it's the end)

In [5]:
time.sleep(3)

elem = driver.find_element(By.TAG_NAME,value="body")

no_of_pagedowns = 30

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    no_of_pagedowns-=1

#### Get HTML

In [47]:
html_txt=driver.page_source
soup=BeautifulSoup(html_txt)
viviendas= soup.find('section',class_='re-SearchResult')

In [48]:
lista_precios=[]
lista_titulos=[]
lista_habitaciones=[]
lista_baños=[]
lista_superficie=[]
lista_planta=[]
lista_ascensor=[]
lista_links=[]
lista_publicacion=[]
lista_fechas=[]


#### Some operation to avoid ads

In [49]:
vivienda_str_list=[]

for vivienda in viviendas:     
    vivienda_str_len=len(str(vivienda))
    #print(vivienda_str_len)
    vivienda_str_list.append(vivienda_str_len)
    
mean_len=statistics.mean(vivienda_str_list)*0.80
print('this is mean_list:',mean_len)

this is mean_list: 6374.8848484848495


#### Get precios

In [50]:
for vivienda in viviendas:     
    if len(str(vivienda))<mean_len:
        pass
    
    else:
        precio=vivienda.find('span',class_='re-CardPrice').getText()
        lista_precios.append(precio)
        

 
print(lista_precios)
print(len(lista_precios))

['97.000 €', '56.000 €', '80.000 €', '65.000 €', '95.000 €', '45.000 €', '200.000 €', '99.000 €', '89.000 €', '110.000 €', '125.000 €', '115.000 €', '130.000 €', '139.000 €', '75.000 €', '125.000 €', '145.000 €', '145.000 €', '120.000 €', '266.000 €', '58.000 €', '95.900 €', '135.000 €', '120.000 €', '199.900 €', '170.000 €', '150.000 €', '59.000 €', '215.000 €', '53.000 €']
30


#### Get Título

In [51]:
for vivienda in viviendas:     
    if len(str(vivienda))<mean_len:
        pass
    else: 
        titulos=vivienda.find('span',class_='re-CardTitle').getText()
        lista_titulos.append(titulos)
        

print(lista_titulos)
print(len(lista_titulos))

['Piso con terraza en Peñalba, San Esteban ', 'Piso con calefacción en San Esteban ', 'Apartamento con terraza en San Esteban ', 'Casa o chalet con 2 baños en Dama de Arintero, San Esteban ', 'Apartamento con terraza en Calle Peña Labra, San Esteban ', 'Piso con 1 baño en Calle León XIII, San Esteban ', 'Piso con terraza en Calle San Juan de Prado, San Esteban ', 'Piso con terraza en San Esteban ', 'Apartamento con ascensor en San Esteban ', 'Piso con ascensor en Mariano Andrés, San Esteban ', 'Piso con terraza en Calle la Cepeda, San Esteban ', 'Piso con balcón en Padre Isla, San Esteban ', 'Piso con ascensor en Calle Cornion, San Esteban ', 'Piso con ascensor en San Esteban ', 'Piso con terraza en Peña Castro, San Esteban ', 'Piso con terraza en San Esteban ', 'Dúplex con 2 baños en Gumersindo Azcarate, San Esteban ', 'Dúplex con 2 baños en Gumersindo Azcarate, San Esteban ', 'Dúplex con terraza en San Esteban ', 'Piso con terraza en San Esteban ', 'Apartamento con ascensor en San Es

#### Get Habitaciones

In [52]:
for vivienda in viviendas:     
    vivienda_str=str(vivienda)
    hab_pos=re.search(r'habs.',vivienda_str)
    hab_pos_indi=re.search(r'hab.',vivienda_str)
    
    if len(str(vivienda))<mean_len:
        pass
    
    elif hab_pos:
        hab_pos_text=hab_pos.start()
        hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
        lista_habitaciones.append(hab)
    else:
        hab_pos_text=hab_pos_indi.start()
        hab=vivienda_str[hab_pos_text-2:hab_pos_text-1]
        lista_habitaciones.append(hab)       

 
print(lista_habitaciones)
print(len(lista_habitaciones))


['3', '3', '2', '4', '2', '2', '4', '3', '2', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '2', '2', '3', '4', '3', '5', '5', '3', '2', '4', '1']
30


#### Get Baños

In [53]:
for vivienda in viviendas:     
    vivienda_str=str(vivienda)
     
    
    if len(str(vivienda))<mean_len:
        pass
    
    else:
        
        baño_pos=re.search(r'(baño|Baño)',vivienda_str)
        baño_pos_text=baño_pos.start()
        b=vivienda_str[baño_pos_text-2:baño_pos_text-1]
        lista_baños.append(b)
        

 
print(lista_baños)
print(len(lista_baños))


['1', '1', '1', '2', '1', '1', '2', '2', '1', '1', '1', '1', '2', '2', '1', '2', '2', '2', '2', '2', '2', '1', '2', '2', '3', '2', '2', '1', '2', '1']
30


#### Get Superficie

In [54]:
for vivienda in viviendas:     
    vivienda_str=str(vivienda)
     
    
    if len(str(vivienda))<mean_len:
        pass
    
    else:
        
        superficie_pos=re.search(r'([0-9] [m])',vivienda_str)
        superficie_pos_text=superficie_pos.start()
        s=vivienda_str[superficie_pos_text-2:superficie_pos_text+3]

        if 'm' not in s:
            lista_superficie.append(None)
        else:
            lista_superficie.append(re.findall(r'\d+',s)[0])
        

 
print(lista_superficie)
print(len(lista_superficie))

['102', '83', '95', '140', '70', '82', '110', '110', '70', '90', '85', '85', '104', '123', '78', '109', '102', '102', '115', '129', '60', '85', '100', '115', '213', '142', '136', '60', '138', '52']
30


#### Get Ascensor

In [55]:
for vivienda in viviendas:     
    vivienda_str=str(vivienda)
    keyword='ascensor'
    
    if len(str(vivienda))<mean_len:
        pass
    
    elif keyword in vivienda_str:
        lista_ascensor.append('SI')
        
    else:
        lista_ascensor.append('NO')
        

 
print(lista_ascensor)
print(len(lista_ascensor))

['SI', 'NO', 'SI', 'NO', 'SI', 'NO', 'SI', 'NO', 'SI', 'SI', 'SI', 'SI', 'SI', 'SI', 'NO', 'SI', 'NO', 'NO', 'SI', 'SI', 'SI', 'SI', 'SI', 'SI', 'NO', 'SI', 'SI', 'NO', 'NO', 'NO']
30


#### Get Planta

In [56]:
for vivienda in viviendas:     
    vivienda_str=str(vivienda)
    planta='Planta'
    planta_baja='planta'
    
    if len(str(vivienda))<mean_len:
        pass
    
    elif planta in vivienda_str:
        planta_pos=re.search(r'(Planta)',vivienda_str)
        planta_pos_text=planta_pos.start()
        p=vivienda_str[planta_pos_text-3:planta_pos_text-2]
        lista_planta.append(p)        
    
    elif planta_baja in vivienda_str:
        planta_pos_baja=re.search(r'(planta)',vivienda_str)
        planta_pos_text_baja=planta_pos_baja.start()
        lista_planta.append(0)        
    
    else:
        lista_planta.append(None)
        

 
print(lista_planta)
print(len(lista_planta))
    
    
    

['4', '4', '4', 0, '3', '2', '7', '1', '4', '4', '4', '3', '3', '2', '4', '2', '1', '1', 0, 0, '2', '5', '4', '3', '\r', '1', '7', None, 0, '2']
30


#### Get Link

In [57]:
for vivienda in viviendas:     
    if len(str(vivienda))<mean_len:
        pass
    else:
        links = vivienda.find('a',href=True)
        href = links['href']
        fotocasa = 'https://www.fotocasa.es/'
        if '/es/comprar/vivienda/' in href:
            lista_links.append(fotocasa+href)

    
print(lista_links)
print(len(lista_links))

['https://www.fotocasa.es//es/comprar/vivienda/leon-capital/calefaccion-parking-terraza-ascensor/163198397/d?from=list', 'https://www.fotocasa.es//es/comprar/vivienda/leon-capital/calefaccion/165755393/d?from=list', 'https://www.fotocasa.es//es/comprar/vivienda/leon-capital/calefaccion-parking-terraza-ascensor/176536123/d?from=list', 'https://www.fotocasa.es//es/comprar/vivienda/leon-capital/jardin-no-amueblado/155990685/d?from=list', 'https://www.fotocasa.es//es/comprar/vivienda/leon-capital/calefaccion-terraza-trastero-ascensor-parking-internet/165606125/d?from=list', 'https://www.fotocasa.es//es/comprar/vivienda/leon-capital/san-esteban/176308126/d?from=list', 'https://www.fotocasa.es//es/comprar/vivienda/leon-capital/calefaccion-terraza-ascensor-amueblado-parking-internet/164335226/d?from=list', 'https://www.fotocasa.es//es/comprar/vivienda/leon-capital/calefaccion-parking-terraza-trastero/161445633/d?from=list', 'https://www.fotocasa.es//es/comprar/vivienda/leon-capital/calefaccio

#### Get Fecha Publicación

In [58]:
for vivienda in viviendas:     
    if len(str(vivienda))<mean_len:
        pass
    
    else:
        fecha=vivienda.find('span',class_='re-CardTimeAgo').getText()
        if re.search(r'\d+', fecha):
            fecha_int=int(re.search(r'\d+', fecha).group())
        else:
            fecha_int=0
        lista_fechas.append(fecha_int)
 
print(lista_fechas)
print(len(lista_fechas))

[31, 63, 8, 29, 9, 29, 45, 110, 53, 52, 106, 17, 145, 42, 6, 62, 1, 6, 14, 23, 105, 9, 185, 0, 0, 0, 0, 0, 0, 0]
30


## If is not the first time: scroll down

#### Create DataFrame

In [18]:
df=pd.DataFrame(list(zip(lista_titulos,lista_precios,lista_habitaciones,lista_planta,lista_superficie,lista_baños,lista_ascensor,lista_fechas ,lista_links))).drop_duplicates()


In [19]:
df=df.rename(columns={0:"Título",1:"Precio",2:"Habitaciones",3:"Planta",4:"Superficie",5:"Baños",6:"Ascensor",7:"Fecha Publicación",8:"Links"})


In [20]:
barrios=['San Esteban','Palomera']
ciudades=['León']
df['Barrio']=barrios[0]
df['Ciudad']=ciudades[0]

#### Next page

In [45]:
time.sleep(5)

WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.')))).click()


In [24]:
'''elem_up = driver.find_element(By.TAG_NAME,value="body")

no_of_pagedowns = 30

while no_of_pagedowns:
    elem_up.send_keys(Keys.PAGE_UP+Keys.COMMAND)
    time.sleep(0.25)
    no_of_pagedowns-=1'''

'elem_up = driver.find_element(By.TAG_NAME,value="body")\n\nno_of_pagedowns = 30\n\nwhile no_of_pagedowns:\n    elem_up.send_keys(Keys.PAGE_UP+Keys.COMMAND)\n    time.sleep(0.25)\n    no_of_pagedowns-=1'

#### Scroll down again

In [46]:
time.sleep(3)

elem = driver.find_element(By.TAG_NAME,value="body")

no_of_pagedowns = 30

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    no_of_pagedowns-=1

#### Go back to fill the values of the lists

#### Create NEW DataFrame to then merge to the new one

In [59]:
n_df=pd.DataFrame(list(zip(lista_titulos,lista_precios,lista_habitaciones,lista_planta,lista_superficie,lista_baños,lista_ascensor,lista_fechas ,lista_links))).drop_duplicates()

n_df=df.rename(columns={0:"Título",1:"Precio",2:"Habitaciones",3:"Planta",4:"Superficie",5:"Baños",6:"Ascensor",7:"Fecha Publicación",8:"Links"})

n_df['Barrio']=barrios[0]
n_df['Ciudad']=ciudades[0]

In [60]:
df = pd.concat([df,n_df])


In [61]:
df.drop_duplicates()

,Título,Precio,Habitaciones,Planta,Superficie,Baños,Ascensor,Fecha Publicación,Links,Barrio,Ciudad
0,"Piso con terraza en Peñalba, San Esteban",97.000 €,3,4,102,1,SI,31,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
1,Piso con calefacción en San Esteban,56.000 €,3,4,83,1,NO,63,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
2,Apartamento con terraza en San Esteban,80.000 €,2,4,95,1,SI,8,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
3,"Casa o chalet con 2 baños en Dama de Arintero,...",65.000 €,4,0,140,2,NO,29,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
4,"Apartamento con terraza en Calle Peña Labra, S...",95.000 €,2,3,70,1,SI,9,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
5,"Piso con 1 baño en Calle León XIII, San Esteban",45.000 €,2,2,82,1,NO,29,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
6,"Piso con terraza en Calle San Juan de Prado, S...",200.000 €,4,7,110,2,SI,45,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
7,Piso con terraza en San Esteban,99.000 €,3,1,110,2,NO,110,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
8,Apartamento con ascensor en San Esteban,89.000 €,2,4,70,1,SI,53,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León
9,"Piso con ascensor en Mariano Andrés, San Esteban",110.000 €,3,4,90,1,SI,52,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban,León


In [62]:
len(df)

120

In [63]:
df.to_csv('for_sale.csv',index=False)